##Hago dataset de mentira y tomo de a partes para entregarle al modelo


In [18]:
import random
import torch
import numpy as np
import matplotlib.pyplot as plt
from torch.utils import data

In [19]:
def synthetic_data(w, b, num_examples): 
    """Generate y = Xw + b + noise.
    num_ examples = numero de registros de mi dataset o numero de muestras
    w = lista de parametros, uno por cada feature.
    b = parametro termino independiente
    Estos seran los parametros verdaderos que mi modelo debera predecir"""
    
    #Genero numeros aleatorios con distribucion gaussiana (parametros 0 y 1), estos numeros seran un array de filas = num_examples columnas = len(w)
    X = torch.normal(0, 1, (num_examples, len(w)))

    #Mediante la funcion genero los resultados de y exactos, sera una matriz de 1 x num_examples 
    y = torch.matmul(X, w) + b

    #Introduzco ruido a los resultados
    y += torch.normal(0, 0.01, y.shape)

    #Devuelvo las muestras X y hago un reshape a y para que pase de ser 1 x num_examples a num_examples x 1
    return X, y.reshape((-1, 1))

true_w = torch.tensor([2, -3.4])
true_b = 4.2
features, labels = synthetic_data(true_w, true_b, 1000)

In [20]:
features.shape

torch.Size([1000, 2])

In [21]:
labels.shape

torch.Size([1000, 1])

In [22]:
def load_array(data_arrays, batch_size, is_train=True): 
    """Construct a PyTorch data iterator."""
    dataset = data.TensorDataset(*data_arrays)
    #genero un iterable sobre el dataset
    return data.DataLoader(dataset, batch_size, shuffle=is_train)
    #El shuffle hace que cada vez que se llama desordena todo, esto para no pasarle siempre el mismo orden de registros

In [23]:
batch_size = 10
data_iter = load_array((features, labels), batch_size)

In [24]:
#Me dara una lista donde el primer elemento son las filas y el segundo el target, el numero de filas sera controlado por el tamaño del batch
next(iter(data_iter))

[tensor([[ 2.3919, -1.5827],
         [-0.3891, -1.1564],
         [-0.4609,  0.9488],
         [ 0.9481,  0.6297],
         [-0.7618,  0.5512],
         [-0.2562, -1.2905],
         [ 1.3527,  0.1660],
         [-1.2684,  0.2662],
         [-0.5053,  0.9694],
         [-0.0452, -1.3483]]), tensor([[14.3500],
         [ 7.3547],
         [ 0.0695],
         [ 3.9446],
         [ 0.7961],
         [ 8.0728],
         [ 6.3311],
         [ 0.7522],
         [-0.1039],
         [ 8.7021]])]

##Defino el modelo

In [25]:
from torch import nn
#nn.sequencial es una forma de hacer redes neuronales, en este caso le estoy diciendo que mi red va a ser la mas basica de todas:
#es decir le voy a meter dos variables a una sola neurona (la neurona hace la regresion lineal)
net = nn.Sequential(nn.Linear(2, 1))

##Inicializo los parametros del modelo

In [26]:
#net[0] porque quiero acceder a la primera capa de la red neuronal (en nuestro caso la unica)
net[0].weight.data.normal_(0, 0.01) #los pesos w seran una distribucion gaussiana
net[0].bias.data.fill_(0) # el b lo igualare a cero

tensor([0.])

##Definiendo la funcion de perdida

In [27]:
## se utilizara el error cuadratico medio
loss = nn.MSELoss()

##Definiendo el algoritmo de optimizacion de la funcion de perdida

In [28]:
#lr = learning rate
# se usara el Stochastic gradient descent
trainer = torch.optim.SGD(net.parameters(), lr=0.0003)

##Entrenamiento

En cada epoca se pasara enteramente por todo el data set, agarrando de a partes (batch), para cada uno de estos pedazos se realizara:
* Generar predicciones llamando a la neurona net y calcularemos la funcion de perdida
* Calculamos los gradientes con backpropagation
* Actualizamos los parametros con el descenso del gradiente

Cada fin de epoca se computa la funcion de perdida para ver su progreso.

In [29]:
num_epochs = 30
for epoch in range(num_epochs):
    for X, y in data_iter:
        l = loss(net(X), y)
        trainer.zero_grad()
        l.backward()
        trainer.step()
    l = loss(net(features), labels)
    print(f'epoch {epoch + 1}, loss {l:f}')

epoch 1, loss 28.280796
epoch 2, loss 25.215179
epoch 3, loss 22.482016
epoch 4, loss 20.045265
epoch 5, loss 17.872679
epoch 6, loss 15.935685
epoch 7, loss 14.208711
epoch 8, loss 12.668974
epoch 9, loss 11.296176
epoch 10, loss 10.072170
epoch 11, loss 8.980846
epoch 12, loss 8.007823
epoch 13, loss 7.140266
epoch 14, loss 6.366745
epoch 15, loss 5.677042
epoch 16, loss 5.062093
epoch 17, loss 4.513781
epoch 18, loss 4.024875
epoch 19, loss 3.588945
epoch 20, loss 3.200251
epoch 21, loss 2.853673
epoch 22, loss 2.544647
epoch 23, loss 2.269098
epoch 24, loss 2.023399
epoch 25, loss 1.804318
epoch 26, loss 1.608965
epoch 27, loss 1.434776
epoch 28, loss 1.279453
epoch 29, loss 1.140949
epoch 30, loss 1.017442


In [30]:
#Error en la estimacion

w = net[0].weight.data
print('error in estimating w:', true_w - w.reshape(true_w.shape))
b = net[0].bias.data
print('error in estimating b:', true_b - b)

error in estimating w: tensor([ 0.3637, -0.6357])
error in estimating b: tensor([0.7286])
